In [1]:
import os

result_dir = "/media/ajad/YourBook/AshokSaugatResearchBackup/AshokSaugatResearch/steerable-scene-generation/outputs/2025-10-12"

In [2]:
from PIL import Image
import matplotlib.pyplot as plt

In [8]:
def compare_results(result_dir, id=None):
    from PIL import ImageOps

    fig, ax = plt.subplots(2, 2, figsize=(20, 12))  # Made figure size significantly bigger
    file_name = None

    # Adjust padding to better fit the larger figure, could be tweaked as needed
    plt.subplots_adjust(wspace=0.2, hspace=0.3, left=0.07, right=0.95, top=0.85, bottom=0.05)

    border_width = 8  # You can increase or decrease as necessary
    border_color = 'black'  # Color of the border

    for file in os.listdir(os.path.join(result_dir, "baseline_ddpm")):
        if file.endswith(".png"):
            # Plot image
            if id is not None:
                if str(id) in file:
                    file_name = file
                    img = Image.open(os.path.join(result_dir, "baseline_ddpm", file))
                    img = ImageOps.expand(img, border=border_width, fill=border_color)
                    ax[0, 0].imshow(img)
                    ax[0, 0].set_title("baseline_ddpm", pad=12)
                    ax[0, 0].axis("off")
                    break
    for file in os.listdir(os.path.join(result_dir, "baseline_ddim")):
        if file.endswith(".png"):
            # Plot image
            if id is not None:
                if str(id) in file:
                    img = Image.open(os.path.join(result_dir, "baseline_ddim", file))
                    img = ImageOps.expand(img, border=border_width, fill=border_color)
                    ax[0, 1].imshow(img)
                    ax[0, 1].set_title("baseline_ddim", pad=12)
                    ax[0, 1].axis("off")
                    break
    for file in os.listdir(os.path.join(result_dir, "rl_ddpm")):
        if file.endswith(".png"):
            # Plot image
            if id is not None:
                if str(id) in file:
                    img = Image.open(os.path.join(result_dir, "rl_ddpm", file))
                    img = ImageOps.expand(img, border=border_width, fill=border_color)
                    ax[1, 0].imshow(img)
                    ax[1, 0].set_title("rl_ddpm", pad=12)
                    ax[1, 0].axis("off")
                    break
    for file in os.listdir(os.path.join(result_dir, "rl_ddim")):
        if file.endswith(".png"):
            # Plot image
            if id is not None:
                if str(id) in file:
                    img = Image.open(os.path.join(result_dir, "rl_ddim", file))
                    img = ImageOps.expand(img, border=border_width, fill=border_color)
                    ax[1, 1].imshow(img)
                    ax[1, 1].set_title("rl_ddim", pad=12)
                    ax[1, 1].axis("off")
                    break

    # Add extra padding above the suptitle
    fig.suptitle(f"Comparison of {file_name}", fontsize=16, y=1.05)
    save_dir = "/media/ajad/YourBook/AshokSaugatResearchBackup/AshokSaugatResearch/steerable-scene-generation/comparisons"
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(os.path.join(save_dir, f"comparison_{file_name}"))
    plt.close()
    # plt.show()

# compare_results(result_dir, "0000")

for i in range(100):
    compare_results(result_dir, str(i).zfill(4))
